In [ ]:
from benchmark import Benchmark

bm = Benchmark('Fresnel Propagation', 'N', [2**i for i in range(6, 13)], t_min=0.3)

In [ ]:
# Physical parameters
diameter = 20e-3  # m
pupil_diameter = 10e-3
wavelength = 632e-9 # m
distance = 1  # m

# Numerical parameters
N = 512

In [ ]:
import hcipy

def fresnel_hcipy(it, N):
    pupil_grid = hcipy.make_pupil_grid(N, diameter=diameter)

    aperture = hcipy.make_circular_aperture(diameter=pupil_diameter)(pupil_grid)
    wf = hcipy.Wavefront(aperture, wavelength)

    prop = hcipy.FresnelPropagator(pupil_grid, distance, num_oversampling=1, zero_padding=1)

    for _ in it:
        wf_out = prop(wf)

    return wf_out.intensity

bm.run(fresnel_hcipy, 'hcipy')

In [ ]:
from prysm.coordinates import make_xy_grid, cart_to_polar
from prysm.geometry import circle
from prysm.propagation import Wavefront, angular_spectrum_transfer_function

def fresnel_prysm(it, N):
    xi, eta = make_xy_grid(N, diameter=diameter * 1e3)

    r, t = cart_to_polar(xi, eta)
    A = circle(pupil_diameter * 1e3 / 2, r)

    dx = xi[0, 1] - xi[0, 0]

    wf = Wavefront.from_amp_and_phase(A, None, wavelength * 1e6, dx)

    tf = angular_spectrum_transfer_function(N, wavelength * 1e6, dx, distance)

    for _ in it:
        wt_out = wf.free_space(tf=tf)

bm.run(fresnel_prysm, 'prysm')

In [ ]:
import poppy
import astropy.units as u

def fresnel_poppy(it, N):
    wf = poppy.FresnelWavefront(diameter * u.m, wavelength=wavelength, npix=N, oversample=1)
    wf *= poppy.CircularAperture(radius=pupil_diameter / 2)

    for _ in it:
        wf2 = wf.copy()
        wf2.propagate_fresnel(distance)

    return wf2.intensity

bm.run(fresnel_poppy, 'poppy')

In [ ]:
bm.plot()